# Potential Energy of system of water molecules

In [24]:
import numpy as np
import re
import random

We get the charges of molecules from the psf file

In [25]:
with open("ICES.psf" , "r") as f:
    data = f.readlines()
    
q = []
num_mols = []
mols = []

for i in data[6:1302]:
    i = i.strip()
    
    mol = re.sub('\s+' , ' ' , i).split(' ')[5]
    ch = float(re.sub('\s+' , ' ' , i).split(' ')[6])
    num = int(re.sub('\s+' , ' ' , i).split(' ')[2])
    
    mols.append(mol)
    q.append(ch)
    num_mols.append(num)

We get the configuration of the water molecules from the pdb files 

In [26]:
with open("starting_config_300k.pdb" , "r") as f:
    data = f.readlines()
X = []
Y = []
Z = []

# Hard coding the lengths
L = [23.623, 22.406, 27.1759]

for i in data[:-1]:
    i = i.strip()
    x = float(re.sub('\s+' , ' ' , i).split(' ')[5])
    y = float(re.sub('\s+' , ' ' , i).split(' ')[6])
    z = float(re.sub('\s+' , ' ' , i).split(' ')[7])
    X.append(x)
    Y.append(y)
    Z.append(z)

In [27]:
k = 332.1
A = 582000.0
B = 595.0

In [28]:
def calc_dist(x1, y1, z1, x2, y2, z2):
    dd = 0
    
    rx = abs(x2 - x1)
    rx -= L[0] * (round(rx/L[0]))
    dd += rx**2
    
    ry = abs(y2 - y1)
    ry -= L[1] * (round(ry/L[1]))
    dd += ry**2

    rz = abs(z2 - z1)
    rz -= L[2] * (round(rz/L[2]))
    dd += rz**2
    dd = np.sqrt(dd)
    return dd

def calc_q(q1, q2, r):
    q = k
    q *= q1*q2
    q /= r
    return q

In [29]:
E = 0
DIST = []
for i in range(1296):
    energy = 0
    Q = q[i]
    x = X[i]
    y = Y[i]
    z = Z[i]
    num_of_mmol = num_mols[i]
    for j in range(i+1,1296):
        ch = q[j]
        xc = X[j]
        yc = Y[j]
        zc = Z[j]
        r = calc_dist(x, y, z, xc, yc, zc)
        
        mn = num_mols[j]
        if mn == num_of_mmol:
            continue
        
        DIST.append(r)
        energy += calc_q(Q, ch, r)
        
        if mols[j] == mols[i] and mols[i] == 'OT':
            energy += ((A/(r**12)) - (B/(r**6)))
    E += energy
        
print('Total Potential Energy of system:', E)

Total Potential Energy of system: -4559.667472442068
